In [23]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [24]:
num_epochs = 100
total_series_length = 50000
truncated_backprop_length = 15
state_size = 4
num_classes = 2
echo_step = 3
batch_size = 5
num_batches = total_series_length//batch_size//truncated_backprop_length

In [25]:
def generateData():
    x = np.array(np.random.choice(2, total_series_length, p=[0.5, 0.5]))
    y = np.roll(x, echo_step)
    y[0:echo_step] = 0

    x = x.reshape((batch_size, -1))  # The first index changing slowest, subseries as rows
    y = y.reshape((batch_size, -1))

    return (x, y)

In [26]:
x,y = generateData()

In [27]:
y

array([[0, 0, 0, ..., 1, 0, 1],
       [1, 0, 1, ..., 0, 1, 1],
       [1, 1, 0, ..., 0, 1, 0],
       [0, 0, 1, ..., 0, 1, 0],
       [1, 1, 1, ..., 0, 0, 0]])

In [28]:
batchX_placeholder = tf.placeholder(tf.float32, [batch_size, truncated_backprop_length])
batchY_placeholder = tf.placeholder(tf.int32, [batch_size, truncated_backprop_length])

In [29]:
init_state = tf.placeholder(tf.float32, [batch_size, state_size])

In [30]:
W2 = tf.Variable(np.random.rand(state_size, num_classes),dtype=tf.float32)
b2 = tf.Variable(np.zeros((1,num_classes)), dtype=tf.float32)

In [31]:
# Unstack columns
inputs_series = tf.split(batchX_placeholder, truncated_backprop_length, 1)
labels_series = tf.unstack(batchY_placeholder, axis=1)

In [32]:
# Forward passes
cell = tf.nn.rnn_cell.BasicRNNCell(state_size)

In [33]:
with tf.variable_scope('rnn'):
    states_series, current_state = tf.contrib.rnn.static_rnn(cell, inputs_series, init_state)

In [34]:
logits_series = [tf.matmul(state, W2) + b2 for state in states_series] #Broadcasted addition
predictions_series = [tf.nn.softmax(logits) for logits in logits_series]

losses = [tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels) for logits, 
          labels in zip(logits_series, labels_series)]
total_loss = tf.reduce_mean(losses)

In [35]:
train_step = tf.train.AdagradOptimizer(0.3).minimize(total_loss)

In [ ]:
def plot(loss_list, predictions_series, batchX, batchY):
    plt.subplot(2, 3, 1)
    plt.cla()
    plt.plot(loss_list)

    for batch_series_idx in range(5):
        one_hot_output_series = np.array(predictions_series)[:, batch_series_idx, :]
        single_output_series = np.array([(1 if out[0] < 0.5 else 0) for out in one_hot_output_series])

        plt.subplot(2, 3, batch_series_idx + 2)
        plt.cla()
        plt.axis([0, truncated_backprop_length, 0, 2])
        left_offset = range(truncated_backprop_length)
        plt.bar(left_offset, batchX[batch_series_idx, :], width=1, color="blue")
        plt.bar(left_offset, batchY[batch_series_idx, :] * 0.5, width=1, color="red")
        plt.bar(left_offset, single_output_series * 0.3, width=1, color="green")

    plt.draw()
    plt.pause(0.0001)

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    plt.ion()
    plt.figure()
    plt.show()
    loss_list = []

    for epoch_idx in range(num_epochs):
        x,y = generateData()
        _current_state = np.zeros((batch_size, state_size))

        print("New data, epoch", epoch_idx)

        for batch_idx in range(num_batches):
            start_idx = batch_idx * truncated_backprop_length
            end_idx = start_idx + truncated_backprop_length

            batchX = x[:,start_idx:end_idx]
            batchY = y[:,start_idx:end_idx]

            _total_loss, _train_step, _current_state, _predictions_series = sess.run(
                [total_loss, train_step, current_state, predictions_series],
                feed_dict={
                    batchX_placeholder:batchX,
                    batchY_placeholder:batchY,
                    init_state:_current_state
                })

            loss_list.append(_total_loss)

            if batch_idx%100 == 0:
                print("Step",batch_idx, "Loss", _total_loss)
                plot(loss_list, _predictions_series, batchX, batchY)

New data, epoch 0
Step 0 Loss 0.676896
Step 100 Loss 0.0100653
Step 200 Loss 0.00419869
Step 300 Loss 0.00286022
Step 400 Loss 0.00189963
Step 500 Loss 0.00150061
Step 600 Loss 0.00137031
New data, epoch 1
Step 0 Loss 0.147484
Step 100 Loss 0.00101968
Step 200 Loss 0.00091393
Step 300 Loss 0.000710553
Step 400 Loss 0.000753431
Step 500 Loss 0.000635459
Step 600 Loss 0.000620349
New data, epoch 2
Step 0 Loss 0.24802
Step 100 Loss 0.000549947
Step 200 Loss 0.000519671
Step 300 Loss 0.000437783
Step 400 Loss 0.000428243
Step 500 Loss 0.000391896
Step 600 Loss 0.000417159
New data, epoch 3
Step 0 Loss 0.215999
Step 100 Loss 0.000355656
Step 200 Loss 0.000357785
Step 300 Loss 0.000345312
Step 400 Loss 0.000328897
Step 500 Loss 0.000318686
Step 600 Loss 0.000284867
New data, epoch 4
Step 0 Loss 0.543779
Step 100 Loss 0.000878374
Step 200 Loss 0.000790111
Step 300 Loss 0.000674697
Step 400 Loss 0.000557391
Step 500 Loss 0.0005387
Step 600 Loss 0.000486981
New data, epoch 5
Step 0 Loss 0.17952

In [ ]:
plt.ioff()
plt.show()